In [1]:
import copy
import os
import tempfile
# import unittest

import numpy as np
# import pytest
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, AutoConfig, LlamaConfig, LlamaForCausalLM

from trl import SFTTrainer
from trl.import_utils import is_peft_available
from trl.trainer import ConstantLengthDataset, DataCollatorForCompletionOnlyLM


tmp_dir = './temp_dir'
# model_id = "trl-internal-testing/dummy-GPT2-correct-vocab"
# model = AutoModelForCausalLM.from_pretrained(model_id)

model_id = 'HuggingFaceM4/tiny-random-LlamaForCausalLM'
model = AutoModelForCausalLM.from_pretrained(model_id)
print(model)
print(model.config)

/Users/denghang/Miniconda3/envs/llm/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 16, padding_idx=31999)
    (layers): ModuleList(
      (0-1): 2 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=16, out_features=16, bias=False)
          (k_proj): Linear(in_features=16, out_features=16, bias=False)
          (v_proj): Linear(in_features=16, out_features=16, bias=False)
          (o_proj): Linear(in_features=16, out_features=16, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=16, out_features=64, bias=False)
          (up_proj): Linear(in_features=16, out_features=64, bias=False)
          (down_proj): Linear(in_features=64, out_features=16, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=16, out_features=32000, bias=False)
)

LlamaConfig {
  "_name_or_path": "HuggingFaceM4/tiny-random-LlamaForCausalLM",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "eos_token_id": 1,
  "hidden_act": "silu",
  "hidden_size": 16,
  "initializer_range": 0.02,
  "intermediate_size": 64,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 4,
  "num_hidden_layers": 2,
  "num_key_value_heads": 4,
  "pad_token_id": -1,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.38.1",
  "use_cache": true,
  "vocab_size": 32000
}

In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_id, add_special_tokens=False)
print(tokenizer)

LlamaTokenizerFast(name_or_path='HuggingFaceM4/tiny-random-LlamaForCausalLM', vocab_size=32000, 
model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': 
'<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  
added_tokens_decoder={
        0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

# 模型增加Pad Token

In [3]:
model.pad_token_id = tokenizer.pad_token_id
model.pad_token = tokenizer.pad_token

print(model)
print(tokenizer)
print(tokenizer('<unk>', add_special_tokens = False))

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 16, padding_idx=31999)
    (layers): ModuleList(
      (0-1): 2 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=16, out_features=16, bias=False)
          (k_proj): Linear(in_features=16, out_features=16, bias=False)
          (v_proj): Linear(in_features=16, out_features=16, bias=False)
          (o_proj): Linear(in_features=16, out_features=16, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=16, out_features=64, bias=False)
          (up_proj): Linear(in_features=16, out_features=64, bias=False)
          (down_proj): Linear(in_features=64, out_features=16, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=16, out_features=32000, bias=False)
)

LlamaTokenizerFast(name_or_path='HuggingFaceM4/tiny-random-LlamaForCausalLM', vocab_size=32000, 
model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': 
'<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  
added_tokens_decoder={
        0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

{'input_ids': [0], 'attention_mask': [1]}

# 创建数据集

## 单轮数据

In [4]:
dummy_dataset = Dataset.from_dict(
            {
                "question": [
                    "Does llamas know how to code?",
                    "Does llamas know how to fly?",
                    "Does llamas know how to talk?",
                    "Does llamas know how to code?",
                    "Does llamas know how to fly?",
                    "Does llamas know how to talk?",
                    "Does llamas know how to swim?",
                ],
                "answer": [
                    "Yes, llamas are very good at coding.",
                    "No, llamas can't fly.",
                    "Yes, llamas are very good at talking.",
                    "Yes, llamas are very good at coding.",
                    "No, llamas can't fly.",
                    "Yes, llamas are very good at talking.",
                    "No, llamas can't swim.",
                ],
                # "text": [
                #     "### Question: Does llamas know how to code?\n ### Answer: Yes, llamas are very good at coding.",
                #     "### Question: Does llamas know how to fly?\n ### Answer: No, llamas can't fly.",
                #     "### Question: Does llamas know how to talk?\n ### Answer: Yes, llamas are very good at talking.",
                #     "### Question: Does llamas know how to code?\n ### Answer: Yes, llamas are very good at coding.",
                #     "### Question: Does llamas know how to fly?\n ### Answer: No, llamas can't fly.",
                #     "### Question: Does llamas know how to talk?\n ### Answer: Yes, llamas are very good at talking.",
                #     "### Question: Does llamas know how to swim?\n ### Answer: No, llamas can't swim.",
                # ],
            }
        )

dummy_dataset


Dataset({
│   features: ['question', 'answer'],
│   num_rows: 7
})

# TODO： 多轮数据

# 创建Formating函数

In [5]:
def formatting_prompts_func(example):
    text = f"\n###Question: {example['question']}\n###Answer: {example['answer']}"
    return text


def formatting_prompts_func_batched(example):
    output_text = []
    for i, question in enumerate(example["question"]):
        text = f" \n###Question: {question}\n###Answer: {example['answer'][i]} "
        output_text.append(text)
    return output_text


# 创建collator

In [6]:
training_args = TrainingArguments(
                output_dir=tmp_dir,
                dataloader_drop_last=True,
                evaluation_strategy="no",
                max_steps=512,
                eval_steps=2,
                save_steps=2,
                logging_steps=32,
                per_device_train_batch_size=4,
                learning_rate = 1e-4,
            )

instruction_template = "###Question:"
assistant_template = "###Answer:"

inst_tokenizerd = tokenizer(instruction_template, add_special_tokens=False)
ass_tokenizerd = tokenizer(assistant_template, add_special_tokens=False)

data_collator = DataCollatorForCompletionOnlyLM(
            response_template=ass_tokenizerd['input_ids'][1:], 
            tokenizer=tokenizer, mlm=False)

In [7]:
print(tokenizer.decode([835]))
print(tokenizer.decode([16492]))
print(tokenizer.decode([29901]))

print(tokenizer.decode([835]))
print(tokenizer.decode([22550]))
print(tokenizer.decode([29901]))


print(tokenizer.decode([22550]))
print(tokenizer.decode([673]))

###

Question

:

###

Answer

:

Answer

Answer

# 创建 Trainer

In [8]:
sft_trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dummy_dataset,
    data_collator = data_collator,
    max_seq_length=256,  # make sure there is at least 1 packed sequence
    packing=False, 
    # dataset_text_field = None,
    formatting_func = formatting_prompts_func_batched,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

/Users/denghang/Miniconda3/envs/llm/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


# Debug Collator

In [9]:
print(sft_trainer.train_dataset)
print(sft_trainer.data_collator)
# print(tokenizer(assistant_template))

batch = data_collator([sft_trainer.train_dataset[0]['input_ids'], sft_trainer.train_dataset[1]['input_ids']])
print(batch)

print(tokenizer.decode(sft_trainer.train_dataset[0]['input_ids'], skip_special_tokens=False))
print(tokenizer.decode(sft_trainer.train_dataset[1]['input_ids'], skip_special_tokens=False))


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 7
})

DataCollatorForCompletionOnlyLM(
    tokenizer=LlamaTokenizerFast(name_or_path='HuggingFaceM4/tiny-random-LlamaForCausalLM', vocab_size=32000, 
model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': 
'<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  
added_tokens_decoder={
        0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
},
    mlm=False,
    mlm_probability=0.15,
    pad_to_multiple_of=None,
    tf_experimental_compile=False,
    return_tensors='pt'
)

{
    'input_ids': tensor([[    1,   259,    13,  2277, 29937, 16492, 29901,  5538, 11829,   294,
          1073,   920,   304,   775, 29973,    13,  2277, 29937, 22550, 29901,
          3869, 29892, 11829,   294,   526,  1407,  1781,   472, 14137, 29889,
         29871],
        [    0,     1,   259,    13,  2277, 29937, 16492, 29901,  5538, 11829,
           294,  1073,   920,   304, 11340, 29973,    13,  2277, 29937, 22550,
         29901,  1939, 29892, 11829,   294,   508, 29915, 29873, 11340, 29889,
         29871]]),
    'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          3869, 29892, 11829,   294,   526,  1407,  1781,   472, 14137, 29889,
         29871],
        [ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  1939, 29892, 11829,   294,   508, 29915, 29873, 11340, 29889,
         29871]])
}

<s>  
###Question: Does llamas know how to code?
###Answer: Yes, llamas are very good at coding.

<s>  
###Question: Does llamas know how to fly?
###Answer: No, llamas can't fly.

# sft 训练

In [10]:
# sft_trainer.train()